In [1]:
import scipy

In [2]:
scipy.__version__

'1.2.0.dev0+10233db'

In [3]:
import kwant

import numpy as np
import sympy
sympy.init_printing(print_builtin=False)

# prepare test Hamiltonian

In [4]:
tmp = "alpha_{0} * k_{0} * sigma_{0}"
sx, sy, sz = sympy.symbols('sigma_x sigma_y sigma_z')

ham1 = kwant.continuum.sympify(
    " + ".join(tmp.format(s) for s in ['x', 'y', 'z']),
    locals={'sigma_x': sx, 'sigma_y': sy, 'sigma_z': sz}

)

ham2 = kwant.continuum.sympify(" + ".join(tmp.format(s) for s in ['x', 'y', 'z']))

In [5]:
ham1

In [6]:
ham2

⎡     α_z⋅k_z       αₓ⋅kₓ - ⅈ⋅α_y⋅k_y⎤
⎢                                    ⎥
⎣αₓ⋅kₓ + ⅈ⋅α_y⋅k_y      -α_z⋅k_z     ⎦

# Define rotation

In [7]:
import scipy.linalg as la
from scipy.spatial.transform import Rotation

In [8]:
R = np.array([[0, -1, 0],
              [+1, 0, 0],
              [0, 0, 1]])

# R = kwant.rmt.circular(3, 'BDI')

# n = Rotation.from_dcm(R).as_rotvec()

In [9]:
R @ [1, 0, 0]

array([0, 1, 0])

In [10]:
la.det(R)

1.0

# Define rotation procedure

In [11]:
from semicon.misc import rotate_symbols, basis_rotation

In [12]:
def tmp_rotate(expr, R):
    def rotation_subs(R, v):
        return {cprime: c for (cprime, c) in zip(v, R @ v)}

    subs = rotation_subs(R, sympy.Matrix(kwant.continuum.momentum_operators))
    subs.update(rotation_subs(R, sympy.Matrix([sx, sy, sz])))

    return expr.subs(subs, simultaneous=True).expand()


In [13]:
ham1

In [14]:
s = tmp_rotate(ham1, R); s

In [15]:
kwant.continuum.sympify(str(s))

⎡     α_z⋅k_z       -ⅈ⋅αₓ⋅k_y + α_y⋅kₓ⎤
⎢                                     ⎥
⎣ⅈ⋅αₓ⋅k_y + α_y⋅kₓ       -α_z⋅k_z     ⎦

In [16]:
from semicon.models import spin_operators
from semicon.misc import rotate_symbols, basis_rotation, round_sympy

In [17]:
S = kwant.continuum.sympify("[[sigma_x, sigma_y, sigma_z]]") / 2

t = rotate_symbols(ham2, R)
U = basis_rotation(R, S)

In [18]:
t = round_sympy(U @ t @ U.transpose().conjugate()).expand()

# test 

In [19]:
round_sympy(sympy.expand(kwant.continuum.sympify(str(s)) - round_sympy(t)))

⎡0  0⎤
⎢    ⎥
⎣0  0⎦

# on kp ham

In [20]:
import semicon
bands = ('gamma_8v', )
kpham = semicon.models.foreman(bands=bands)
S = semicon.models.spin_operators(bands=bands)

In [21]:
from semicon.kp_models.explicit_zeeman import Jx, Jy, Jz

In [22]:
t = rotate_symbols(kpham, R)
U = basis_rotation(R, S)

t = (U @ t @ U.transpose().conjugate()).expand()

In [23]:
sympy.sympify(U)

⎡-0.707106781186547 + 0.707106781186548⋅ⅈ                     0               
⎢                                                                             
⎢                   0                      0.707106781186548 + 0.7071067811865
⎢                                                                             
⎢                   0                                         0               
⎢                                                                             
⎣                   0                                         0               

                         0                                        0           
                                                                              
47⋅ⅈ                     0                                        0           
                                                                              
      0.707106781186548 - 0.707106781186547⋅ⅈ                     0           
                                                   

In [24]:
t = round_sympy(t, 15)

In [25]:
t[0, 1]